In [1]:
!pip install apache-tvm

In [2]:
import tvm
from tvm import te
from tvm import autotvm
import tvm.testing
import numpy as np
from tvm.autotvm.tuner import XGBTuner, GATuner, RandomTuner, GridSearchTuner

import logging
import sys

In [3]:
target = 'llvm'
dev = tvm.cpu(0)
dtype="float32"
M, N, K = 1024, 1024, 1024
A_ = np.random.uniform(size=(N, K))
B_ = np.random.uniform(size=(K, M))

In [4]:
# @autotvm.template("tvm_my_matmul")
def tvm_my_matmul_transform():
    A = te.placeholder((M, K), name='A')
    B = te.placeholder((K, N), name='B')

    k = te.reduce_axis((0, K), name='k')
    C = te.compute((M, N),
                    lambda i, j: te.sum(
                        A[i, k] * B[k, j], axis=k
                        ), name='C')

    s = te.create_schedule(C.op)

    i, j = C.op.axis
    io, ii = s[C].split(i, factor=4)
    jo, ji = s[C].split(j, factor=4)
    return s, [A, B, C]

s, (A, B, C) = tvm_my_matmul_transform()
print(tvm.lower(s, [A, B, C], simple_mode=True))

@main = primfn(A_1: handle, B_1: handle, C_1: handle) -> ()
  attr = {"from_legacy_te_schedule": True, "global_symbol": "main", "tir.noalias": True}
  buffers = {A: Buffer(A_2: Pointer(float32), float32, [1024, 1024], []),
             B: Buffer(B_2: Pointer(float32), float32, [1024, 1024], []),
             C: Buffer(C_2: Pointer(float32), float32, [1024, 1024], [])}
  buffer_map = {A_1: A, B_1: B, C_1: C} {
  for (i.outer: int32, 0, 256) {
    for (i.inner: int32, 0, 4) {
      for (j.outer: int32, 0, 256) {
        for (j.inner: int32, 0, 4) {
          C_3: Buffer(C_2, float32, [1048576], [])[((((i.outer*4096) + (i.inner*1024)) + (j.outer*4)) + j.inner)] = 0f32
          for (k: int32, 0, 1024) {
            let cse_var_3: int32 = (j.outer*4)
            let cse_var_2: int32 = ((i.outer*4096) + (i.inner*1024))
            let cse_var_1: int32 = ((cse_var_2 + cse_var_3) + j.inner)
            C_3[cse_var_1] = (C_3[cse_var_1] + (A_3: Buffer(A_2, float32, [1048576], [])[(cse_var_2 + k

In [5]:
def run_tuning(tasks, measure_option, tuner="gridsearch", early_stopping=None,
               log_filename="tuning.log", n_trial=None):
    for i, task in enumerate(tasks):
        prefix = "[Task %2d/%2d] " % (i + 1, len(tasks))

        # create tuner
        if tuner == "xgb" or tuner == "xgb-rank":
            tuner_obj = XGBTuner(task, loss_type="rank")
        elif tuner == "ga":
            tuner_obj = GATuner(task, pop_size=50)
        elif tuner == "random":
            tuner_obj = RandomTuner(task)
        elif tuner == "gridsearch":
            tuner_obj = GridSearchTuner(task)
        else:
            raise ValueError("Invalid tuner: " + tuner)

        # do tuning
        n_trial = min(n_trial or len(task.config_space), len(task.config_space))
        tuner_obj.tune(
            n_trial=n_trial or len(task.config_space),
            early_stopping=early_stopping,
            measure_option=measure_option,
            callbacks=[
                autotvm.callback.progress_bar(n_trial, prefix=prefix),
                autotvm.callback.log_to_file(log_filename),
            ],
        )

In [6]:
logging.getLogger("autotvm").setLevel(logging.DEBUG)
logging.getLogger("autotvm").addHandler(logging.StreamHandler(sys.stdout))

log_file = "tune.log"
tuning_option = {
    "log_filename": log_file,
    "n_trial": 10,
    "tuner": "xgb",
    "early_stopping": None,
    "measure_option": autotvm.measure_option(
        builder='local',
        runner=autotvm.LocalRunner(
            number=5, repeat=2, min_repeat_ms=0, enable_cpu_cache_flush=True
        ),
    ),
}


In [7]:
@autotvm.template("my_matmul_template")
def tvm_my_matmul_template(M, N, K, dtype):
    A = te.placeholder((M, K), name='A', dtype=dtype)
    B = te.placeholder((K, N), name='B', dtype=dtype)

    k = te.reduce_axis((0, K), name='k')
    C = te.compute((M, N),
                    lambda i, j: te.sum(
                        A[i, k] * B[k, j], axis=k
                        ), name='C')

    s = te.create_schedule(C.op)

    i, j = s[C].op.axis
    k = s[C].op.reduce_axis[0]

    candidates = [[1, 1024], [2, 512], [4, 256], [8, 128], [16, 64], [32, 32], [64, 16], [128, 8], [256, 4], [512, 2], [1024, 1]]

    cfg = autotvm.get_config()
    cfg.define_split("tile_y", i, num_outputs=2, policy="candidate", candidate=candidates)
    cfg.define_split("tile_x", j, num_outputs=2, policy="candidate", candidate=candidates)

    yo, yi = cfg["tile_y"].apply(s, C, i)
    xo, xi = cfg["tile_x"].apply(s, C, j)

    s[C].reorder(yo, xo, k, yi, xi)

    return s, [A, B, C]

In [8]:
!lscpu | grep L

Byte Order:                         Little Endian
L1d cache:                          32 KiB (1 instance)
L1i cache:                          32 KiB (1 instance)
L2 cache:                           256 KiB (1 instance)
L3 cache:                           55 MiB (1 instance)
Vulnerability L1tf:                 Mitigation; PTE Inversion


In [9]:
def filter(e):
    max_bx = e["tile_x"].size[0] <= e["tile_x"].size[1]
    inline_cache = e["tile_y"].size[1] <= e["tile_x"].size[1]
    cache_kbytes = 32
    constrains = 4 * (e["tile_x"].size[1] + e["tile_y"].size[1] + e["tile_x"].size[1] * e["tile_y"].size[1]) <= cache_kbytes * 1024
    return max_bx and constrains

In [10]:
@autotvm.template("filtered_c_template")
def filtered_c_template(M, N, K, dtype):
    A = te.placeholder((M, K), name='A', dtype=dtype)
    B = te.placeholder((K, N), name='B', dtype=dtype)

    k = te.reduce_axis((0, K), name='k')
    C = te.compute((M, N),
                    lambda i, j: te.sum(
                        A[i, k] * B[k, j], axis=k
                        ), name='C')

    s = te.create_schedule(C.op)

    i, j = s[C].op.axis
    k = s[C].op.reduce_axis[0]

    candidates = [[1, 1024], [2, 512], [4, 256], [8, 128], [16, 64], [32, 32], [64, 16], [128, 8], [256, 4], [512, 2], [1024, 1]]

    cfg = autotvm.get_config()
    cfg.multi_filter(filter=filter)
    cfg.define_split("tile_y", i, num_outputs=2, policy="candidate", candidate=candidates)
    cfg.define_split("tile_x", j, num_outputs=2, policy="candidate", candidate=candidates)

    yo, yi = cfg["tile_y"].apply(s, C, i)
    xo, xi = cfg["tile_x"].apply(s, C, j)

    s[C].reorder(yo, xo, k, yi, xi)

    return s, [A, B, C]

In [11]:
task1 = autotvm.task.create("my_matmul_template", args=(M, N, K, dtype), target=target)
print(task1.config_space)
# for idx in range(task1.config_space.range_length):
#     if task1.config_space.is_index_valid(idx):
#         print(task1.config_space.get(idx))

ConfigSpace (len=121, range_length=121, space_map=
   0 tile_y: Split(policy=candidate, product=1024, num_outputs=2) len=11
   1 tile_x: Split(policy=candidate, product=1024, num_outputs=2) len=11
)


In [12]:
task2 = autotvm.task.create("filtered_c_template", args=(M, N, K, dtype), target=target)
print(task2.config_space)
# for idx in range(task2.config_space.range_length):
#     if task2.config_space.is_index_valid(idx):
#         print(task2.config_space.get(idx))

ConfigSpace (len=33, range_length=121, space_map=
   0 tile_y: Split(policy=candidate, product=1024, num_outputs=2) len=11
   1 tile_x: Split(policy=candidate, product=1024, num_outputs=2) len=11
)


In [ ]:
tasks = [task1, task2]
run_tuning(tasks, **tuning_option)

In [15]:
from collections import defaultdict
import json
results = defaultdict(list)
with open(log_file, 'r') as f:
    data = f.read()
    data = data.split('\n')
    for d in data:
        if not d:
            continue
        d = json.loads(d)
        name = d['input'][1]
        sz = d['config']['entity']
        res = d['result']
        res_v = res[-2]
        res_e = res[-3]
        if res_e == 0:
            results[name].append((res_v, sz))


for name, res in results.items():
    print(name, sorted(res, key=lambda x: x[0])[:2])

my_matmul_template [(2.9015676975250244, [['tile_y', 'sp', [128, 8]], ['tile_x', 'sp', [2, 512]]]), (3.1571240425109863, [['tile_y', 'sp', [8, 128]], ['tile_x', 'sp', [1, 1024]]])]
filtered_c_template [(2.9636354446411133, [['tile_y', 'sp', [1024, 1]], ['tile_x', 'sp', [1, 1024]]]), (3.156834125518799, [['tile_y', 'sp', [512, 2]], ['tile_x', 'sp', [4, 256]]])]


In [16]:
dev = tvm.cpu(0)
a = np.random.uniform(size=(M, K)).astype(dtype)
b = np.random.uniform(size=(K, N)).astype(dtype)
c_tvm = tvm.nd.array(np.zeros((M, N), dtype=dtype), dev)
c_np = np.matmul(a, b)
a = tvm.nd.array(a)
b = tvm.nd.array(b)

In [17]:
with autotvm.apply_history_best(log_file):
    with tvm.target.Target(target):
        s, (A, B, C) = tvm_my_matmul_template(M, N, K, dtype)
        func = tvm.build(s, [A, B, C])
        print(tvm.lower(s, [A, B, C], simple_mode=True))

        func(a, b, c_tvm)
        tvm.testing.assert_allclose(c_np, c_tvm.numpy(), rtol=1e-4)

        time_f = func.time_evaluator(func.entry_name, dev, number=10)
        cost = time_f(a,b,c_tvm).mean
        print("func", cost*1000)

Finish loading 20 records


DEBUG:autotvm:Finish loading 20 records


@main = primfn(A_1: handle, B_1: handle, C_1: handle) -> ()
  attr = {"from_legacy_te_schedule": True, "global_symbol": "main", "tir.noalias": True}
  buffers = {A: Buffer(A_2: Pointer(float32), float32, [1024, 1024], []),
             B: Buffer(B_2: Pointer(float32), float32, [1024, 1024], []),
             C: Buffer(C_2: Pointer(float32), float32, [1024, 1024], [])}
  buffer_map = {A_1: A, B_1: B, C_1: C} {
  for (i.outer: int32, 0, 128) {
    for (j.outer: int32, 0, 2) {
      for (i.inner.init: int32, 0, 8) {
        for (j.inner.init: int32, 0, 512) {
          C_3: Buffer(C_2, float32, [1048576], [])[((((i.outer*8192) + (i.inner.init*1024)) + (j.outer*512)) + j.inner.init)] = 0f32
        }
      }
      for (k: int32, 0, 1024) {
        for (i.inner: int32, 0, 8) {
          for (j.inner: int32, 0, 512) {
            let cse_var_3: int32 = (j.outer*512)
            let cse_var_2: int32 = ((i.outer*8192) + (i.inner*1024))
            let cse_var_1: int32 = ((cse_var_2 + cse_var_3

In [18]:
with autotvm.apply_history_best(log_file):
    with tvm.target.Target(target):
        s, (A, B, C) = filtered_c_template(M, N, K, dtype)
        func = tvm.build(s, [A, B, C])
        print(tvm.lower(s, [A, B, C], simple_mode=True))

        func(a, b, c_tvm)
        tvm.testing.assert_allclose(c_np, c_tvm.numpy(), rtol=1e-4)

        time_f = func.time_evaluator(func.entry_name, dev, number=10)
        cost = time_f(a,b,c_tvm).mean
        print("func", cost*1000)

Finish loading 20 records


DEBUG:autotvm:Finish loading 20 records


@main = primfn(A_1: handle, B_1: handle, C_1: handle) -> ()
  attr = {"from_legacy_te_schedule": True, "global_symbol": "main", "tir.noalias": True}
  buffers = {A: Buffer(A_2: Pointer(float32), float32, [1024, 1024], []),
             B: Buffer(B_2: Pointer(float32), float32, [1024, 1024], []),
             C: Buffer(C_2: Pointer(float32), float32, [1024, 1024], [])}
  buffer_map = {A_1: A, B_1: B, C_1: C} {
  for (i.outer: int32, 0, 1024) {
    for (j.inner.init: int32, 0, 1024) {
      C_3: Buffer(C_2, float32, [1048576], [])[((i.outer*1024) + j.inner.init)] = 0f32
    }
    for (k: int32, 0, 1024) {
      for (j.inner: int32, 0, 1024) {
        let cse_var_2: int32 = (i.outer*1024)
        let cse_var_1: int32 = (cse_var_2 + j.inner)
        C_3[cse_var_1] = (C_3[cse_var_1] + (A_3: Buffer(A_2, float32, [1048576], [])[(cse_var_2 + k)]*B_3: Buffer(B_2, float32, [1048576], [])[((k*1024) + j.inner)]))
      }
    }
  }
}


func 245.2255676


Io, Ii - e["tile_y"].size[0], e["tile_y"].size[1]

Jo, Ji - e["tile_x"].size[0], e["tile_x"].size[1]

- Базовый оптимизация - увеличение размеров блоков, т.е Ji и Ii должны быть больше, чем обратные им Jo и Io соответственно. (Io <= Ii, Jo <= Ji)
- Причем желательно, чтобы Ji >= Ii, т.к это длина последовательных элементов в памяти и к ним обращение будет оптимальнее.
- Для ограничения размеров блоков сверху выберем одну итерацию k, мы хотим чтобы блоки матриц A, B, C, с которыми мы работаем на этой итерации содержались в кеше. Их размеры соответственно: Ii, Ji, Ii*Ji. Т.к кеш у этой машины 32 килобайта, и мы работаем c float32 (4 байта), то получается соотношение 4 * (Ii + Ji + Ii*Ji) <= 32 * 1024
- Чтобы сильно не сужать пространство поиска уберем не самое важное условие Io <= Ii

При проведении исследований в таргет не включались векторные расширения AVX для чистоты экспериментов